<a href="https://colab.research.google.com/github/Sammypacoo/Rosalind/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import plotly.graph_objects as go

categories = ['GNOMAD', 'SABE']

fig = go.Figure()


fig.add_trace(go.Scatterpolar(
      r=[12.9, 9.09],
      theta=categories,
      fill='toself',
      name='APP'
))
fig.add_trace(go.Scatterpolar(
      r=[27.8, 19.3],
      theta=categories,
      fill='toself',
      name='MAPT'
))

fig.add_trace(go.Scatterpolar(
      r=[13,0],
      theta=categories,
      fill='toself',
      name='PSEN1'
))
fig.add_trace(go.Scatterpolar(
      r=[10.4,0],
      theta=categories,
      fill='toself',
      name='PSEN2'
))
fig.add_trace(go.Scatterpolar(
      r=[ 6.2,5],
      theta=categories,
      fill='toself',
      name='TREM2'
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 50]
    )),
  showlegend=True
)

fig.show()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from collections import defaultdict
import gzip
import pandas as pd
import re

#Funções do código


In [7]:
def Convert(tup, di):
    for a, b in tup:
        di.setdefault(a, []).append(b)
    return di
      
# Driver Code    
tups = [("akash", 10), ("gaurav", 12), ("anand", 14), 
     ("suraj", 20), ("akhil", 25), ("ashish", 30)]
dictionary = {}
print (Convert(tups, dictionary))

{'akash': [10], 'gaurav': [12], 'anand': [14], 'suraj': [20], 'akhil': [25], 'ashish': [30]}


In [8]:

variantes=pd.read_csv("/content/drive/MyDrive/Projeto/Projeto/Arquivos_do_projeto/app_gnomad.csv", sep=';')
variantes.columns



arq1=pd.read_csv("/content/drive/MyDrive/Projeto/Projeto/Arquivos_importantes_p_codigo/CCDS.current.txt" , sep='\t')


arq2=arquivo=pd.read_csv("/content/drive/MyDrive/Projeto/Projeto/Arquivos_importantes_p_codigo/CCDS2Sequence.current.txt" , sep='\t')


arq3=pd.read_csv("/content/drive/MyDrive/Projeto/Projeto/Arquivos_importantes_p_codigo/exon_seq1", header= None , sep='\t')
# criei uma tabela com os dados CCDS_protein_exons.current.faa"

df=pd.read_table("/content/drive/MyDrive/Projeto/Projeto/Arquivos_importantes_p_codigo/GRCh38_latest_genomic.gtf",header=None,skiprows=4)



In [9]:


def tab_final(gene):

    arq1.shape
    arq1.rename(columns={"ccds_id":"#ccds"}, inplace = True)
    arq1.columns
    app=arq1[arq1["gene"]==gene]
    app.shape
    app.columns
    app

    result = pd.merge(app, arq2, on="#ccds",how="left")
    result = result.drop_duplicates()
    result=result[result["source"]=="NCBI"]
    result=result[result["status_in_CCDS"]=="Accepted"]
    result
    return result     

In [10]:
# Para cada posição do genomica, mostra tods os NM relacionados aquela posi genomica
def tab_teste1(final,x):
  x=int(x)

  # Fazer listas das extremidades do gene

  ext1=final.iloc[:,7].tolist()
  ext2=final.iloc[:,8].tolist()
  lista_fil1=[]
  for j in range (len(ext1)):
      x1=ext1[j]
      x1=int(x1)+1
      x2=ext2[j]
      x2=int(x2)+2
      r=range(x1,x2)

      if x in r:
          lista_fil1+=[1]
      else:
          lista_fil1+=[0]
  teste=final
  teste["filtro1"]=lista_fil1
  teste=teste[teste["filtro1"]==1]


  # Verificar em  quais nms aquela posi genômica está presente 
  tam1=teste.shape
  tam1=tam1[0]
  lista_fil2=[]
  for xl in range(tam1):
      f1=teste.iloc[xl,9]
      f1=str(f1)
      f1=re.findall(r'[0-9]+', f1)
      for xl1 in range (0,len(f1)-1,2):
          ini=f1[xl1]
          ini=int(ini)+1
          final=f1[xl1+1]
          final=int(final)+2
          r3=range(ini,final)
          if x in r3:
            lista_fil2+=[1] 
            break
          else:
            if xl1 == len(f1)-2:
                lista_fil2+=[0]
           

  teste["filtro2"]=lista_fil2
  teste1=teste[teste["filtro2"]==1]

     
  #teste1.rename(columns={5:"#ccds"}, inplace = True)
  return teste1

In [11]:
# final seria o teste1
def tab_exons_seq(final):
  taman=final.shape
  taman=taman[0]
  if taman >0:
    df=pd.DataFrame()  
    dic={}
    dic2={}
    g2=[]
    for i in range (taman):
      strand=final.iloc[i,6]
      dic={}
      bb=final.iloc[i,9]
      tf=bb.replace("[","")
      tf=tf.replace("]","")
      tf=tf.split(",")
      for i2 in range(len(tf)):    
        p=tf[i2]
        p=p.split("-")
        p=p[0]
        p=int(p)
        dic[tf[i2]]=p
      d_descending =sorted(dic.items(), key=lambda x: x[1], reverse=True)
      g1=dic.keys()
      g1=list(g1)
      dic2={}
      Convert(d_descending, dic2)
      g2=dic2.keys()
      g2=list(g2)
      g=final.iloc[i,4]
      if strand=="-":
        strand='-'
        y=arq3[arq3[0]==g]
        y["éxon"]=g2
      else:
        strand='+'
        y=arq3[arq3[0]==g]
        y["éxon"]=g1

      
      lin=y.shape
      lin=lin[0]
      list_strand=[]
      for it in range(lin):
          list_strand+=[strand]
      y["strand"]=list_strand
      df=pd.concat([df,y],axis=0)
  else:
     df=pd.DataFrame() 

  return df

In [12]:
def resul(teste1,df,x):
  x=int(x)
  taman=df.shape
  taman=taman[0]
  if taman >0:
    lis=df.iloc[:,0].unique()
    lis=lis.tolist()

    l1=[]
    #colocar o for
    for jkk in range(len(lis)):
      f=df[df[0]==lis[jkk]]
      ggp=f[4].tolist()
      listp=[]
      for j in range (len(ggp)):
        ggp1=len(ggp[j])
        listp+=[ggp1]
      f["tamanho"]=listp

      listp2=[0]
      valor=0
      for j1 in range (len(listp)-1):
        valor=valor + listp[j1]
        listp2+=[valor]
      f["Start_Value"]=listp2
      l=[]    
      for j2 in range (len(listp)):
        ran=f.iloc[j2,5]
        ran=ran.split("-")
        a1=int(ran[0])+1
        b1=int(ran[1])+2
        r=range(a1,b1)
        if x in r:
          l+=[1]
        else:
          l+=[0]

      f["filtro"]=l
      gg3=f[f["filtro"]==1]

      strand=gg3.iloc[0,6]
      ran=gg3.iloc[0,5]

      cc=ran.split("-")
      a1=int(cc[0])+1
      b1=int(cc[1])+1
      

      if strand=="-":
        d=b1-x       
        s=(d//3)+1
      else:
        a1=a1-1
        d=x-a1
        
        s=(d//3)
        

        

      s1=gg3.iloc[0,8]
      
      s,s1
      res=s+s1
      res
      l1+=[res]


    nm=teste1.iloc[:,14]
    nm=nm.tolist()

    gg4=pd.DataFrame(nm,columns=["Nm"])
    gg5=pd.DataFrame(l1,columns=["Pos"])
    gg6=pd.concat([gg4,gg5],axis=1)
    tamk=gg6.shape
    tamk=tamk[0]
    listaf=[]
    for ih in range (tamk):
      it=gg6.iloc[ih,:]
      it=it.tolist()
      listaf+=[it]
  else:
    listaf=[]
  return listaf

In [13]:
import re 

def sitios(X,gene):
    if len(X)>0:
      ptm=pd.read_csv("/content/drive/MyDrive/Projeto/Projeto/Arquivos_do_projeto/Arquivos_importantes/phosphositeplus_allptms_REFSEQ1", sep='\t')
      ptm.columns

      ptm=ptm[ptm["GENE"]==gene]
      ptm.dropna(subset = ["Refseq_semdigito"], inplace=True)
      nm_sitio=ptm["Refseq_semdigito"].tolist()
      tes=ptm["Refseq_semdigito"].unique()
      lista=[]
      for cd in range (len(nm_sitio)):
          x=nm_sitio[cd]
          x=str(x)
          x1=re.findall(r'\bNM_\w+',x)
          x1=x1[0]
          lista+=[x1]
      
      
      ptm["R"]=lista
      pos_sitio=ptm["SITE"].tolist()
      lista2=[]
      for cd1 in range (len(pos_sitio)):
          #print(cd1)
          x=pos_sitio[cd1]
          x=str(x)
          x1=re.findall(r'[0-9]+',x)
          x1=x1[0]
          x1=int(x1)
          #print(x1)
          lista2+=[x1]
      len(lista2)
      ptm["Posição"]=lista2
      s="n"
      lista4=[]
      for i in range (len(X)):
          nm=X[i][0]
          nm=re.findall(r'\bNM_\w+',nm)
          nm=nm[0]
          pos=X[i][1]
          pos=str(pos)
          pos=re.findall(r'[0-9]+', pos)
          pos=pos[0]
          pos=int(pos)
          
          A=pos+1
          a=pos-1
          lista1=[a,pos,A]
          t=-2  
          nm_sitio=ptm[ptm["R"]==nm]
      
          tam=nm_sitio.shape
          tam=tam[0]
          if tam!=0:
              #print("entrei")
                    
              for v in range (len(lista1)):
                  sitio_p=lista1[v]
                  #print(sitio_p)
                  t+=1
      
                  p_sitio=nm_sitio[nm_sitio["Posição"]==sitio_p]
                  #print(p_sitio)
      
                  tam1=p_sitio.shape
                  tam1=tam1[0]
                  if tam1!=0:
                      #print("entrei1")                
                      s="s"
                      tam3=p_sitio.shape
                      tam3=tam3[0]
                      lista3=[]
                      for k in range (tam3):
                          row=p_sitio.iloc[:,[0,1,12,13]]
                          row=row.iloc[k,:].tolist()
                          row+=[t]
                          
                          lista3+=[row]
                      lista4+=[lista3]
                    
                      break

    else : 
      s="n"
      lista4=[]                     
    return s, lista4

#total

In [ ]:
dic={"APP":"/content/drive/MyDrive/Projeto/Projeto/Arquivos_do_projeto/app_gnomad.csv",
     "MAPT":"/content/drive/MyDrive/Projeto/Projeto/MAPT_gnomad.csv",
     "PSEN1":"/content/drive/MyDrive/Projeto/Projeto/PSEN1_gnomad.csv",
     "PSEN2":"/content/drive/MyDrive/Projeto/Projeto/PSEN2_gnomad.csv",
     "TREM2":"/content/drive/MyDrive/Projeto/Projeto/TREM2_gnomad.csv"}
dic_re={}
df_resul_gnomad=pd.DataFrame(columns=["Gene","Gnomad Total","Gnomad_ptm","Gnomad_Porc"])


for chave in dic :
  if chave=="APP":
    variantes=pd.read_csv(dic[chave], sep=';')
  else:
    variantes=pd.read_csv(dic[chave], sep=',')
  gene=chave
  final=tab_final(gene)
  final
  variantes.head()
  variantes=variantes[variantes["VEP Annotation"]=="missense_variant"]
  variantes
  list_x=variantes["Position"].tolist()

  tk=[]
  tk1=[]
  tk2=[]
  for hgh in range(len(list_x)):
    print(hgh)  
    x=list_x[hgh]

    teste1=tab_teste1(final,x)
    teste1

    df=tab_exons_seq(teste1)
    #print(df)
    lista=resul(teste1,df,x)

    a,b=sitios(lista,gene)
    tk+=[a]
    tk1+=[b]
    tk2+=[lista]
  print(tk)
  variantes["PTM"]=tk
  variantes["PTM_INF"]=tk1
  variantes["Variantes"]=tk2
  variantes_gnomad_sabe=variantes
  variantes_sito=variantes[variantes["PTM"]=="s"]
  dic_re[chave]=variantes_sito
  X=len(variantes)
  X1=len(variantes_sito)
  X2=(X1*100)/X
  X,X1,X2
  lg=[chave,X,X1,X2]
  df_resul_gnomad.loc[len(df_resul_gnomad)] = lg
  df_resul_gnomad


In [15]:
  df_resul_gnomad

,Gene,Gnomad Total,Gnomad_ptm,Gnomad_Porc
0,APP,263,34,12.927757
1,MAPT,355,99,27.887324
2,PSEN1,138,18,13.043478
3,PSEN2,191,20,10.471204
4,TREM2,120,6,5.000000


In [21]:
dic_re

{'APP':      Chromosome  Position         rsIDs Reference Alternate          Source  \
 0            21  25881702   rs766455623         T         C  gnomAD Genomes   
 1            21  25881705  rs1300518717         G         T  gnomAD Genomes   
 3            21  25881734  rs1437503550         G         C  gnomAD Genomes   
 29           21  25897632  rs1259157720         C         T  gnomAD Genomes   
 30           21  25897641   rs200084346         T         A  gnomAD Genomes   
 31           21  25897642    rs63750363         C         G  gnomAD Genomes   
 33           21  25897649   rs200260102         G         A  gnomAD Genomes   
 34           21  25897655   rs201104127         A         G  gnomAD Genomes   
 35           21  25897659   rs770773038         T         C  gnomAD Genomes   
 36           21  25897667           NaN         C         G  gnomAD Genomes   
 37           21  25897668  rs1305531440         C         T  gnomAD Genomes   
 54           21  25911849   rs20

In [16]:
from google.colab import files
df_resul_gnomad.to_csv('df_resul_gnomad.csv') 
files.download('df_resul_gnomad.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
variantes=pd.read_csv("/content/drive/MyDrive/Projeto/Projeto/sabe2.txt", sep='\t')
dic_re1={}
df_resul_Sabe=pd.DataFrame(columns=["Gene","Sabe Total","Sabe_ptm","Sabe_Porc"])

for chave in dic :
  gene=chave
  variantes1=variantes[variantes["Gene"]==gene]
  final=tab_final(gene)
  final
  list_x=variantes1["posi"].tolist()
  
  tk=[]
  tk1=[]
  tk2=[]
  for hgh in range(len(list_x)):
    print(hgh)  
    x=list_x[hgh]

    teste1=tab_teste1(final,x)
    teste1
    df=tab_exons_seq(teste1)
    lista=resul(teste1,df,x)

    a,b=sitios(lista,gene)
    tk+=[a]
    tk1+=[b]
    tk2+=[lista]
  print(tk)
  variantes1["PTM"]=tk
  variantes1["PTM_INF"]=tk1
  variantes1["Variantes"]=tk2
  variantes_sito=variantes1[variantes1["PTM"]=="s"]
  dic_re1[chave]=variantes_sito
  X=len(variantes1)
  X1=len(variantes_sito)
  X2=(X1*100)/X
  X,X1,X2
  lg=[chave,X,X1,X2]
  df_resul_Sabe.loc[len(df_resul_Sabe)] = lg
  

In [18]:
df_resul_Sabe

,Gene,Sabe Total,Sabe_ptm,Sabe_Porc
0,APP,11,1,9.090909
1,MAPT,31,6,19.354839
2,PSEN1,4,0,0.000000
3,PSEN2,11,0,0.000000
4,TREM2,16,1,6.250000


In [19]:
from google.colab import files
df_resul_Sabe.to_csv('df_resul_Sabe.csv') 
files.download('df_resul_Sabe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
dic_re1

{'APP':       Chro      posi Ref Al            Conse_P Gene  \
 19398   21  25911855   C  T  nonsynonymous SNV  APP   
 
                                               Transcrip.     avsnp147 FILTER  \
 19398  APP:NM_001136129:exon11:c.G1402A:p.E468K,APP:N...  rs140304729   PASS   
 
       CEGH_Filter  HomozygousALT_count  Hemizygous_count  Allele_number  \
 19398         vSR                    0               0.0           2342   
 
        Allele_ALT_count  Frequencies         Cohort PTM  \
 19398               2.0     0.000854  SABE-WGS-1171   s   
 
                                       PTM_INF  \
 19398  [[[O-GalNAc, PSP, NM_000484, 600, 1]]]   
 
                                                Variantes  
 19398  [[NM_000484.4, 599], [NM_201414.3, 524], [NM_2...  ,
 'MAPT':        Chro      posi Ref Al            Conse_P  Gene  \
 160412   17  45983441   G  A  nonsynonymous SNV  MAPT   
 160415   17  45983912   C  T  nonsynonymous SNV  MAPT   
 160426   17  45971888   G  A  non